In [1]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [4]:
@everywhere function build_controller(scale)
    input_channel = RemoteChannel(() -> Channel{Vector{Float64}}(1), myid())
    output_channel = RemoteChannel(() -> Channel{Vector{Float64}}(1), myid())
    controller = function (y, x)
        y .= x .* scale
#         sleep(1)
    end
    y = zeros(2)
    @async while true
        x = take!(input_channel)
        controller(y, x)
        put!(output_channel, y)
    end
    input_channel, output_channel
end

In [5]:
channels = []

for i in 2:nprocs()
    push!(channels, @fetchfrom(i, build_controller(i)))
end

function parallel_control(channels, x)
    results = Channel{Vector{Float64}}(length(channels))
    @sync begin
        for (input, output) in channels
            @async begin 
                put!(input, x)
                put!(results, take!(output))
            end
        end
    end
    [take!(results) for i in 1:length(channels)]
end

LoadError: [91m[91mStackOverflowError:[39m
deserialize(::Base.Distributed.ClusterSerializer{TCPSocket}, ::Type{RemoteChannel{Channel{Array{Float64,1}}}}) at ./distributed/remotecall.jl:302 (repeats 100 times)[39m

In [58]:
@time parallel_control(channels, [1.0, 2.0])

  0.003203 seconds (669 allocations: 48.594 KiB)


4-element Array{Array{Float64,1},1}:
 [2.0, 4.0] 
 [3.0, 6.0] 
 [4.0, 8.0] 
 [5.0, 10.0]

In [ ]:
@btime @fetch

In [ ]:
using BenchmarkTools

@everywhere function scalar_controller(scale)
    C = RemoteChannel{Channel{Float64}}
    input_channel::C = RemoteChannel(() -> Channel{Float64}(1), myid())
    output_channel::C = RemoteChannel(() -> Channel{Float64}(1), myid())
    controller = let scale = scale
        function (x)
            x * scale
        end
    end
    let input=input_channel, output=output_channel, controller=controller
        @async while true
            x = take!(input)
            y = controller(x)
            put!(output, y)
        end
    end
    input_channel, output_channel
end

channels = NTuple{2, RemoteChannel{Channel{Float64}}}[]

for i in 2:nprocs()
    push!(channels, @fetchfrom(i, scalar_controller(i)))
end

function scalar_parallel_control(channels, x)
    @sync begin
        for (input, output) in channels
            let input=input, output=output, x=x
                @async begin 
                    put!(input, x)
                    take!(output)
                end
            end
        end
    end
end

@btime scalar_parallel_control($channels, 1.0) 

In [7]:
@code_warntype scalar_parallel_control(channels, 1.0)

Variables:
  #self# <optimized out>
  channels::Array{Tuple{RemoteChannel{Channel{Float64}},RemoteChannel{Channel{Float64}}},1}
  x::Float64
  #43::##43#44{Float64,RemoteChannel{Channel{Float64}},RemoteChannel{Channel{Float64}}}
  output@_5 <optimized out>
  input@_6 <optimized out>
  input@_7::RemoteChannel{Channel{Float64}}
  output@_8::RemoteChannel{Channel{Float64}}
  #temp#@_9 <optimized out>
  #temp#@_10::Int64
  v <optimized out>
  t::Task

Body:
  begin 
      # meta: location task.jl # line 301:
      $(Expr(:inbounds, false))
      # meta: location task.jl sync_begin 261
      SSAValue(11) = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), Array{Any,1}, 0, 0, 0))
      SSAValue(9) = $(Expr(:invoke, MethodInstance for get_task_tls(::Task), :(Base.get_task_tls), :($(Expr(:foreigncall, :(:jl_get_current_task), Ref{Task}, svec())))))
      SSAValue(12) = (Core.tuple)(SSAValue(11), $(Expr(:foreigncall, :(:jl_eqtable_get), Any, svec(Any, Any, Any), :((Cor

In [62]:
x = SharedVector{Float64}(2; pids=1:nprocs(), init=true)

2-element SharedArray{Float64,1}:
 0.0
 0.0

In [ ]:
function sa_controller(scale, x)
    